# Final TP

- XGBoost and AdaBoost

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import datetime
import scipy
import mne

from collections import Counter
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from mne.datasets.sleep_physionet.age import fetch_data

import epoch

In [2]:
"""
psg_data
    The psg_data variable contains polysomnography data, which typically includes multiple physiological signals such as EEG, EOG, and EMG, recorded during sleep.

hypnogram_data
    The hypnogram_data variable represents annotations or labels associated with sleep stages, providing information about the temporal distribution of wakefulness, different sleep stages (NREM and REM), and transitions during a sleep recording.
    
fs 
    represents the sampling frequency (or sampling rate) of the polysomnography (PSG) data, indicating the number of samples per second in the signal.
"""

# Reading data (psg) and hypnogram (labels)
psg_file = "data/SC4001E0-PSG.edf"
hypnogram_file = "data/SC4001EC-Hypnogram.edf"

# psg_data = mne.io.read_raw_edf(psg_file)
# hypnogram_data = mne.read_annotations(hypnogram_file)

NameError: name 'psg_data' is not defined

In [4]:
s = epoch.sleepRecording()
s.init_from_file("data/SC4001E0-PSG.edf", "data/SC4001EC-Hypnogram.edf")

Extracting EDF parameters from /Users/stig/Desktop/- TU:e/5Q1 ITBA Buenos Aires/72.75 Aprendizarje Automatico/72.75-ML/final/data/SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/Users/stig/Desktop/- TU:e/5Q1 ITBA Buenos Aires/72.75 Aprendizarje Automatico/72.75-ML/final/epoch.py:62: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  psg_data_raw.set_annotations(hypnogramm_annotations)


In [5]:
# Checking how much of each label we have in the dataset

label_list = [s.epochs[i].label for i in range(len(s.epochs))]
label_counts = Counter(label_list)
for label, count in label_counts.items():
    print(f"{label}: {count}")

Sleep stage W: 1997
Sleep stage 1: 58
Sleep stage 2: 250
Sleep stage 3: 101
Sleep stage 4: 119
Sleep stage R: 125


In [7]:
s.epochs[10].ch_names

['EEG Fpz-Cz',
 'EEG Pz-Oz',
 'EOG horizontal',
 'Resp oro-nasal',
 'EMG submental',
 'Temp rectal',
 'Event marker']

In [ ]:
# FILTRADO

filtered_item = s.epochs[]

N = 5 # Orden del filtro
wn = [1,10] # Frecuencias de corte del filtro
sos = scipy.signal.butter(N, wn, btype = 'bandpass', output = 'sos', fs = fs) # Diseño el filtro
filtrada = scipy.signal.sosfiltfilt(sos, EEG_Fpz_Cz) # Filtro la señal

plt.figure(figsize=(30,8))
plt.plot(t[epoch_start:epoch_end] ,filtrada[epoch_start:epoch_end])
plt.show()

NameError: name 'EEG_Fpz_Cz' is not defined

# Feature extraction

**Time-Based Features:**
- Duration of sleep cycles.
- Time spent in different sleep phases (light sleep, deep sleep, REM).
- Sleep onset latency (time taken to fall asleep).

**Frequency-Based Features:**
- Heart rate variability during different sleep phases.
- Respiratory rate.

**Statistical Features:**
- Mean, median, and standard deviation of various physiological signals during sleep.
- Signal entropy to capture complexity.

**Temporal Patterns:**
- Trends and patterns in sleep cycles over multiple nights.

**Previous Sleep History:**
- Consider features related to the sleep pattern from the previous night.

**Biometric Data:**
- If available, include biometric data like age, gender, and BMI.